In [ ]:
'''
Doing some simple early prototyping to better understand the work flow and where to add

memory addition is pretty meh, need to try something better
'''

In [ ]:
import utils.prompt_utils as prompt_utils
import utils.prototype_embedding_utils as prototype_embedding_utils
import utils.prototype_llm_utils as prototype_llm_utils
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import os


# import importlib

# # Reload the module
# importlib.reload(module_name)

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnableLambda
from langchain.memory import ConversationBufferMemory

In [ ]:
def create_basic_chain():
    memory = ConversationBufferMemory(
        return_messages=True, output_key="answer", input_key="question"
    )
    loaded_memory = RunnablePassthrough.assign(
        chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
    )
    sample_dataset_file_path = os.path.join(os.getcwd(), 'prototype_data', 'ignored', 'little_women_example.txt')
    retriever = prototype_embedding_utils.run_orchestrator_prototype(sample_dataset_file_path)
    prompt = prompt_utils.get_hub_prompt()
    llm = prototype_llm_utils.get_llm()

    rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | loaded_memory
            | prompt
            | llm
            | StrOutputParser()
        )

    return rag_chain, memory


In [ ]:
test_chain, test_memory = create_basic_chain()

In [ ]:
for chunk in test_chain.stream("What is the weather in Greenland today?"):
    print(chunk, end="", flush=True)

In [ ]:
for chunk in test_chain.stream("Who are Beth's sisters?"):
    print(chunk, end="", flush=True)

In [ ]:
test_memory.save_context({"question":"Who are Beth's sisters?"}, {"answer": "Beth's sisters are Meg, Jo, and Amy."})
test_memory.load_memory_variables({})



In [ ]:
for chunk in test_chain.stream("Who did I just ask about?"):
    print(chunk, end="", flush=True)

In [ ]:
# LLM seems the same
# from langchain_openai import ChatOpenAI

# llm = prototype_llm_utils.get_llm()
# print(llm)

# llm2 = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
# # def get_llm(model_name='gpt-3.5-turbo'):
# #     '''
# #     Obvious to dos here
# #     '''
# #     llm = ChatOpenAI(model_name=model_name, temperature=0)

# #     return llm

# print(llm)
# print(llm2)

In [ ]:
import importlib

# Reload the module
importlib.reload(module_name)

# Import the reloaded module
import module_name
